In [3]:
import numpy as np
import pandas as pd

def valuacion_binomial_americana_np(S, K, r, T, sigma, pasos, tipo='put', mostrar_tabla=False):

    # Parámetros básicos
    dt = T / pasos
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    df = np.exp(-r * dt)

    # Árbol de precios del subyacente
    # Matriz triangular donde cada fila es un paso en el tiempo
    precios = np.zeros((pasos + 1, pasos + 1))
    for i in range(pasos + 1):
        for j in range(i + 1):
            precios[j, i] = S * (u ** j) * (d ** (i - j))

    # Valores de la opción al vencimiento
    if tipo == 'call':
        valores = np.maximum(precios[:, pasos] - K, 0)
    else:
        valores = np.maximum(K - precios[:, pasos], 0)

    # Backward induction
    for i in range(pasos - 1, -1, -1):
        for j in range(i + 1):
            valor_esperado = df * (p * valores[j + 1] + (1 - p) * valores[j])
            if tipo == 'call':
                valor_ejercicio = max(precios[j, i] - K, 0)
            else:
                valor_ejercicio = max(K - precios[j, i], 0)
            valores[j] = max(valor_esperado, valor_ejercicio)

    # Mostrar árbol
    if mostrar_tabla:
        tabla = pd.DataFrame(precios)
        tabla = tabla.replace(0, np.nan)
        print("\nÁrbol de precios del subyacente:")
        print(tabla.round(2))
    
    return valores[0]

# Ejemplo
S = 100
K = 100
r = 0.05
T = 1
sigma = 0.2
pasos = 5

precio_put = valuacion_binomial_americana_np(S, K, r, T, sigma, pasos, tipo='put', mostrar_tabla=True)
print(f"\nPrecio opción PUT americana: {precio_put:.4f}")

precio_call = valuacion_binomial_americana_np(S, K, r, T, sigma, pasos, tipo='call')
print(f"Precio opción CALL americana: {precio_call:.4f}")



Árbol de precios del subyacente:
       0       1       2       3       4       5
0  100.0   91.44   83.62   76.47   69.92   63.94
1    NaN  109.36  100.00   91.44   83.62   76.47
2    NaN     NaN  119.59  109.36  100.00   91.44
3    NaN     NaN     NaN  130.78  119.59  109.36
4    NaN     NaN     NaN     NaN  143.01  130.78
5    NaN     NaN     NaN     NaN     NaN  156.39

Precio opción PUT americana: 6.3678
Precio opción CALL americana: 10.8059
